In [5]:
import pandas as pd
from datasets import Dataset

def tokenize_function(examples):
    # Ensure 'question' and 'answer' are strings
    question = ' '.join(examples['question']) if isinstance(examples['question'], list) else examples['question']
    answer = ' '.join(examples['answer']) if isinstance(examples['answer'], list) else examples['answer']
    
    # Concatenate question and answer with a separator
    return tokenizer(question + ' ' + tokenizer.eos_token + ' ' + answer, padding='max_length', truncation=True)

# Load the CSV file
df = pd.read_csv('training_data.csv')

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

# Function to tokenize the data
def tokenize_function(examples):
    # Concatenate question and answer with a separator
    return tokenizer(examples['question'] + ' ' + tokenizer.eos_token + ' ' + examples['answer'], padding='max_length', truncation=True)

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

from transformers import AutoModelForCausalLM

# Load the model
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=4,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    save_steps=10_000,
    save_total_limit=2,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,  # You might want to split your dataset into train and eval sets
)

trainer.train()

model.save_pretrained('./fine_tuned_distilgpt2')
tokenizer.save_pretrained('./fine_tuned_distilgpt2')


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


Map:   0%|                                                                            | 0/10692 [00:00<?, ? examples/s]


TypeError: can only concatenate list (not "str") to list